# Age Prediction Using 2D CNN

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras import Sequential, layers, losses, callbacks
import numpy as np
import datetime
import matplotlib.pyplot as plt
import scipy.io

In [ ]:
url = "https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/wiki_crop.tar"

dataset = tf.keras.utils.get_file(
    "wiki_crop", url,
    untar=True, cache_dir='',
    cache_subdir=''
)
dataset_dir = os.path.join(os.path.dirname(dataset), 'wiki_crop')

In [ ]:
mat = scipy.io.loadmat(os.path.join(dataset_dir, 'wiki.mat'))
mat

In [ ]:
mat['wiki']['dob'][0][0][0]

In [ ]:
dob = np.vectorize(lambda x: datetime.datetime.fromordinal(x).year)(mat['wiki']['dob'][0][0][0])
photo_taken = mat["wiki"]["photo_taken"][0][0][0]

age = (photo_taken - dob).astype(np.float32)
age

In [ ]:
file_path = np.vectorize(lambda x: os.path.join(dataset_dir, x[0]))(mat['wiki']['full_path'][0][0][0])
file_path

In [ ]:
file_age_ds = tf.data.Dataset.from_tensor_slices((file_path, age))

In [ ]:
def parse_file(filename, label):
    img = tf.io.read_file(filename)
    img = tf.io.decode_jpeg(img, channels=1)
    img = tf.image.resize(img,[256, 256])
    label = label / 100
    return img, tf.expand_dims(label,0)

In [ ]:
image_age_ds = file_age_ds.map(parse_file).shuffle(seed=2, buffer_size=64)
image_age_ds

In [ ]:
dataset_size = image_age_ds.cardinality().numpy()
batch_size = 32

AUTOTUNE = tf.data.AUTOTUNE
train_ds = image_age_ds.take(dataset_size*.6).batch(batch_size).prefetch(AUTOTUNE)
val_ds = image_age_ds.skip(dataset_size*.6).take(dataset_size*.2).batch(batch_size).prefetch(AUTOTUNE)
test_ds = image_age_ds.skip(dataset_size*.8).take(dataset_size*.2).batch(batch_size).prefetch(AUTOTUNE)

In [ ]:
model = Sequential([
    layers.Input((256, 256,1)),
    layers.Rescaling(1/255),
    layers.Conv2D(32, (7, 7), padding="valid"),
    layers.ReLU(),
    layers.MaxPool2D((4,4), strides=4),

    layers.Conv2D(64,(3,3), padding="valid"),
    layers.ReLU(),
    layers.MaxPool2D((4,4), strides=4),

    layers.Conv2D(128,(3,3), padding="valid"),
    layers.ReLU(),
    layers.MaxPool2D((2,2), strides=2),

    layers.Conv2D(256,(1,1), padding="valid"),
    layers.ReLU(),
    layers.MaxPool2D((2, 2), strides=2),


    layers.Flatten(),
    layers.Dense(64),
    layers.ReLU(),
    layers.Dense(1)

])

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer="adam",
    loss=losses.MeanSquaredError(),
    metrics=["MAE"]
)

In [ ]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[
        callbacks.TensorBoard(log_dir="logs/ex5")
    ]
)

In [ ]:
model.evaluate(test_ds)

In [ ]:
image, label = next(iter(val_ds))
images = image.numpy()[:9]

res = model(image)
res = tf.squeeze(res)
label = tf.squeeze(label)

In [ ]:
plt.suptitle("Inference")
for i, (img, r) in enumerate(zip(images, res)):
    plt.subplot(331+i)
    plt.title( f"{label[i]*100:.2f} - {res[i]*100:.2f}")
    plt.imshow(img)
    plt.axis("off")
plt.show()
